In [1]:
sys.path.append("../")

import os
import sys
import json
import numpy as np
import matplotlib.pyplot as plt
import cv2

%load_ext autoreload

In [2]:
os.listdir("..\data")
path = "..\data"
targets = []
for folder in os.listdir(path):
    for f in os.listdir(os.path.join(path, folder)):
        if f.endswith(".pdf") :
            targets.append(os.path.join(path, folder, f))
print(targets)

['..\\data\\pdfs\\005-366-fey-squirrel-and-tomb-guardian-compressed.pdf', '..\\data\\pdfs\\006-366-flying-spider-compressed.pdf', '..\\data\\pdfs\\007-366-giant-wolf-compressed.pdf', '..\\data\\pdfs\\008-366-quogwat-compressed.pdf', '..\\data\\pdfs\\009-366-arnitiko-compressed.pdf', '..\\data\\pdfs\\010-366-waterspout-compressed.pdf', '..\\data\\pdfs\\011-366-gargantuan-hamster_and_squid-compressed.pdf', '..\\data\\pdfs\\012-366-el-cadejo-compressed.pdf', '..\\data\\pdfs\\922210-OUTCLASSED_Expansion__Tashas_Panopticon_of_Nemeses.pdf', '..\\data\\pdfs\\Episode 1 - In Scarlet Flames.pdf', '..\\data\\pdfs\\Warlock 15 - Boss Monsters.pdf']


In [3]:
%autoreload 2

from data_loaders.textract_image_loader import TextractImageLoader
from data_loaders.pdf_loader import PDFLoader
from extractor import StatblockExtractor
from preprocessing.columniser import Columniser
from utils.config import get_config, get_argparser
from utils.logger import get_logger

from fifthedition.creature_printer import pretty_format_creature
from fifthedition.creature_parser import CreatureParser
from fifthedition.creature_schema import CreatureSchema



# Get arguments
parser = get_argparser()
args = parser.parse_args(["..\data\clockwork_drake\clockwork_drake.jpg", "--config", r"..\default.conf", "--cache",  r"..\.cache", "--notebook" ])
print(args)

# Get config file
config = get_config(args.config)

logger = get_logger(args.debug, args.notebook, args.logs)

Namespace(targets=['..\\data\\clockwork_drake\\clockwork_drake.jpg'], source=None, authors=None, url=None, output=None, output_format='default', cache='..\\.cache', config='..\\default.conf', logs=None, debug=False, notebook=True)


In [38]:
se = StatblockExtractor(config, logger)
se.register_data_loader(PDFLoader)
#print(target)

for target in targets[-1:]:
    print(target)
    logger.info("Loading creatures from {}".format(target))
    
    results = se.parse(target, args.output)
    parsed_statblocks, statblocks = results

    num_pages = len(statblocks.keys())
    logger.info("Found {} page{} of statblocks".format(num_pages, 's' if num_pages > 1 else ''))
    
    for page in parsed_statblocks:
        print("Page {}:".format(page))
        for creature in parsed_statblocks[page]:
            print()
            print(pretty_format_creature(creature))
            print()



..\data\pdfs\Warlock 15 - Boss Monsters.pdf


Found unexpected number of options in creature text!
	Huge swarm of Small fiends (devil) - ['Huge', 'Small']
Found unexpected number of options in creature text!
	Huge swarm of Small fiends (devil) - ['swarm', 'fiend']
Failed to validate parsed schema for creature_type. Error was:

Key 'type' error:
Or('any', And(<class 'str'>, Use(<method 'lower' of 'str' objects>), <function enum_str.<locals>.<lambda> at 0x000001D7C88D8040>)) did not validate ''
'any' does not match ''
<lambda>('') should evaluate to True



Page 5:

Catscratch
Huge aberration, chaotic evil
-------------------------------------------------------
Armor Class 13 (natural armor)
Hit Points 76 (8d12+24)
Speed 30 ft.
-------------------------------------------------------

-------------------------------------------------------

-------------------------------------------------------


Page 8:

Vexxeh
Huge fiend, lawful evil
-------------------------------------------------------
Armor Class 15 (natural armor)
Hit Points 105 (10d12+40)
Speed 40 ft.
-------------------------------------------------------

-------------------------------------------------------

-------------------------------------------------------


Page 10:

Gulper Behemoth                                          creature is blinded and restrained, it has total cover
Gargantuan monstrosity, unaligned                        against attacks and other effects outside the behemoth,
-------------------------------------------------------  and it takes 10 (3d6) ac

KeyError: 'size'

In [33]:
from fifthedition.creature_printer import pretty_format_creature
print(pretty_format_creature(results[0][0][0]))

Quogwat                                                  Pollen Release. When the quogwat dies, it explodes in a
Tiny plant, chaotic neutral                              burst of pollen.  Each creature within 5 feet of it    
-------------------------------------------------------  must make a DC 11 Constitution saving throw or fall    
Armor Class 12                                           unconscious for 7 days.  Each day, the creature can    
Hit Points 3 (1d4+1)                                     repeat its saving throw, ending the effect on itself   
Speed 20 ft.                                             with a success.  Otherwise, the creature remains       
-------------------------------------------------------  unconscious until a lesser restoration or similar magic
     STR     DEX     CON     INT     WIS     CHA         is cast upon it.  Once the creature wakes, roll a d20  
   14 (+2)  2 (-4)  7 (-1) 13 (+1) 13 (+1)  7 (-1)       and add 1 for every day the creature wa

In [34]:
results[0][0][0].data

{'name': 'Quogwat',
 'size': 'tiny',
 'creature_type': {'type': 'plant', 'swarm': False},
 'alignment': 'chaotic neutral',
 'ac': [{'ac': 12, 'from': [], 'condition': ''}],
 'hp': {'formula': '1d4+1', 'average': 3},
 'speed': [{'type': 'walk', 'distance': 20, 'measure': 'ft'}],
 'abilities': {'str': 2, 'dex': 14, 'con': 13, 'int': 7, 'wis': 13, 'cha': 7},
 'saves': {'int': 0, 'cha': 0},
 'skills': [{'skill': 'Stealth', 'mod': 4}],
 'senses': [],
 'passive': 11,
 'languages': ['Sylvan'],
 'cr': {'cr': '1/8'},
 'features': [{'title': "Nature's Ward",
   'text': "Any creature who targets the Quogwat with an attack or a harmful spell must first make a DC 11 Wisdom saving throw.  On a failed save, the creature must choose a new target or lose the attack or spell.  The ward doesn't protect the Quogwat from area effects, such as the explosion of a fireball."},
  {'title': 'Pollen Release',
   'text': "When the quogwat dies, it explodes in a burst of pollen.  Each creature within 5 feet of it 

In [ ]:
page.cropbox